In [38]:
# Run for GPU/CUDA ML, comment out if you dont have it configured.
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [39]:
"""
test.ipynb used for testing models with test*.csv

This notebook performs a trailing window on specified time series data with the following parameters:
- window size
- hops
In our context, our data is collected on an average of 1 second per movement. Thus, setting our window
 frame as 1s, with a sampling rate of 50Hz would result in 50 samples per window.
"""

import pandas as pd
import tensorflow as tf
import numpy as np
import xarray

##change sampling parameters here
total_samples = 3100*22*4
window = 100
overlap = 50
num_gestures = 4
data_columns = 6

In [40]:
"""
Sliding window function and params
"""
from tqdm import tqdm
from skimage.util.shape import view_as_windows
import warnings

csv = 'temp_testing_sample4.csv'

total_array = np.empty((0,window,6))

print('start!')
df = pd.read_csv(csv, header=None)
total_array = df.to_numpy()
load_original_arr = total_array.reshape(1, window, 6)
print('ok!')
load_original_arr

start!
ok!


array([[[ 4.4000e-01,  5.0000e-01,  8.6000e-01, -8.6600e+00,
          4.7000e-01,  5.9100e+00],
        [ 4.4000e-01,  5.1000e-01,  8.5000e-01, -8.8300e+00,
         -4.3200e+00,  5.9200e+00],
        [ 4.7000e-01,  5.3000e-01,  8.3000e-01, -1.3740e+01,
         -5.9100e+00,  1.0700e+01],
        [ 4.8000e-01,  5.0000e-01,  8.4000e-01, -1.8520e+01,
         -1.5400e+00,  1.4150e+01],
        [ 4.7000e-01,  4.5000e-01,  8.4000e-01, -1.2370e+01,
          8.0000e-01,  1.2220e+01],
        [ 4.5000e-01,  4.8000e-01,  8.6000e-01, -2.3400e+00,
          1.9800e+00,  8.9600e+00],
        [ 4.6000e-01,  4.7000e-01,  8.5000e-01, -3.0100e+00,
         -5.7000e-01,  7.2100e+00],
        [ 4.6000e-01,  4.5000e-01,  8.4000e-01,  1.1100e+00,
          2.9100e+00,  3.6000e+00],
        [ 4.3000e-01,  4.9000e-01,  8.5000e-01,  3.6700e+00,
         -1.0000e-02,  1.2000e-01],
        [ 4.4000e-01,  5.2000e-01,  8.4000e-01,  1.0100e+00,
         -4.9200e+00,  2.9100e+00],
        [ 4.7000e-01,  5.1000e

In [41]:
load_original_arr.shape

(1, 100, 6)

# Testing the model

In [42]:
from keras.models import load_model

model_name = '../lstm_models/lstm_model_100_50_96%'

model = load_model(model_name)

In [43]:
x_test = load_original_arr

In [44]:
y_pred = model.predict(x_test)

In [45]:
rounded = np.round(y_pred)

print(y_pred[0])
print(rounded[0])

[8.0989784e-04 9.5045972e-01 7.4824660e-05 4.8655592e-02]
[0. 1. 0. 0.]


In [46]:
if num_gestures == 4:
    class_names = ["Buddha clap", "Knob", "Pushback", "Swipe"]
elif num_gestures == 5:
    class_names = ["Buddha clap", "Crank", "Knob", "Pushback", "Swipe"]
elif num_gestures == 7:
    class_names = ["Buddha clap", "Crank left", "Crank right", "Knob left", "Knob right", "Pushback", "Swipe"]

name = np.array(class_names)
print(name)
print(type(name))

['Buddha clap' 'Knob' 'Pushback' 'Swipe']
<class 'numpy.ndarray'>


In [47]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(name.reshape(-1, 1))
y_pred = enc.inverse_transform(rounded)

In [48]:
print(y_pred)

[['Knob']]


In [49]:
output = []

for i in range(len(y_pred)):
        output.append(y_pred[i])
        print("Prediction: %s" % (y_pred[i]))

Prediction: ['Knob']


In [50]:
import csv
from datetime import datetime

fields = ['Predicted']
now = datetime.now()
file = 'output ' + now.strftime("%d-%m-%Y %H-%M-%S") + '.csv'
with open(file, 'w', newline='') as filename:
    write = csv.writer(filename)
    write.writerow(fields)
    write.writerows(output)
